In [ ]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import pyGadget as pyg

In [ ]:
palette = ["#404040", "#4C72B0", "#55A868", "#C44E52", "#8172B2"]
sns.set(context='paper', style='ticks', palette=palette, font='serif', font_scale=1.6)
mpl.rc('text', usetex=True)

In [ ]:
#sns.set_palette(sns.cubehelix_palette(5,start=2.7,rot=0.,gamma=0.9,hue=0.9,light=0.8,dark=0.1,reverse=True))
#sns.set_palette(palette)

In [ ]:
store = pd.HDFStore(os.getenv("HOME")+"/data/stampede/radial_properties.hdf5")
print store.keys()
vanilla = store['vanilla']
j0 = store['xr_tau2_J0']
j1 = store['xr_tau2_J1']
j2 = store['xr_tau2_J2']
j3 = store['xr_tau2_J3']
store.close()
data = {'vanilla':vanilla, 'xr_tau2_J0':j0, 'xr_tau2_J1':j1, 'xr_tau2_J2':j2, 'xr_tau2_J3':j3}
simlist = ['vanilla', 'xr_tau2_J0', 'xr_tau2_J1', 'xr_tau2_J2', 'xr_tau2_J3']

In [ ]:
simname = {'vanilla': r'J = 0',
        'xr_tau2_J0': r'J = J$_{0}$', 
        'xr_tau2_J1': r'J = 10 J$_{0}$', 
        'xr_tau2_J2': r'J = 10$^2$ J$_{0}$',
        'xr_tau2_J3': r'J = 10$^3$ J$_{0}$'}
sink_form = {'vanilla': 355,
          'xr_tau2_J0': 255, 
          'xr_tau2_J1': 169,
          'xr_tau2_J2': 227,
          'xr_tau2_J3': 70}

In [ ]:
vanilla.info()

In [ ]:
sinkt0 = pd.DataFrame()
for sim in data.keys():
    df = data[sim]
    df['sim'] = sim
    df = df[df.snapshot == sink_form[sim]]
    df['mtot'] = df.gMshell.cumsum() / 1.989e33
    sinkt0 = pd.concat((sinkt0, df))
sinkt0['rpc'] = sinkt0.radius / 3.08e18
sinkt0['rau'] = sinkt0.radius / 1.49e13
sinkt0['gMsun'] = sinkt0.gMshell / 1.989e33
sinkt0['vr_kms'] = sinkt0.vr / 1e5
sinkt0['vphi_kms'] = sinkt0.vphi /1e5
sinkt0['ndensity'] = sinkt0.gdensity * pyg.constants.X_h / pyg.constants.m_H
sinkt0['mbe'] = 700 * (sinkt0.tshell/200)**1.5 / np.sqrt(sinkt0.ndensity/1e4)
#sinkt0['Mtot'] = 

In [ ]:
with sns.axes_style("ticks"):
    fig, axarr = plt.subplots(4, 1, sharex=True, figsize=(6,12))

for i, gasprop in enumerate(['vr_kms', 'vphi_kms', 'tshell', 'mbe']):
    for sim in simlist[:-1]:
        df = sinkt0[sinkt0.sim == sim]
        axarr[i].semilogx(df.rau, df[gasprop], ls='-', marker='o', ms=3, label=simname[sim])
axarr[2].set_yscale('log')
axarr[3].set_yscale('log')
axarr[0].set_ylim(-2.4,0)
axarr[1].set_ylim(-1.5,3.9)
axarr[2].set_ylim(80, 1.5e3)
#axarr[3].set_ylim(1e2,1e11)
axarr[-1].set_xlim(1e2, 1e6)
axarr[0].legend(loc=0)

axarr[0].set_ylabel('Radial Velocity [km/s]')
axarr[1].set_ylabel('Rotational Velocity [km/s]')
axarr[2].set_ylabel('Temperature [K]')
axarr[3].set_ylabel('Number Density [cm$^{-3}$]')
axarr[-1].set_xlabel("Radius [AU]")
fig.subplots_adjust(hspace=0.01)
sns.despine()
fig.savefig('figures/radial_bins/radial_profile.png', bbox_inches='tight')

In [ ]:
sinkt0

In [ ]:
with sns.axes_style("white"):
    fig, ax = plt.subplots()

for sim in simlist:
    df = sinkt0[sinkt0.sim == sim]
    l, = ax.plot(df.rau, df.mtot, label=simname[sim], lw=2.5)#, marker='o')
    #ax.plot(df.rau, df.mbe, lw = 1.5, color=l.get_c(), zorder=1, marker='o', ms=3)
ax.set_yscale('log')
ax.set_xscale('log')
#ax.set_xlim(50, 1e4)
#ax.set_ylim(.2, 1e3)
ax.legend(loc=0)

In [ ]:
simV = pyg.sim.Simulation('stampede/vanilla',length='AU', mass='solar', refine=False)
sim0 = pyg.sim.Simulation('stampede/xr_tau2_J0',length='AU', mass='solar', refine=False)
sim1 = pyg.sim.Simulation('stampede/xr_tau2_J1',length='AU', mass='solar', refine=False)
sim2 = pyg.sim.Simulation('stampede/xr_tau2_J2',length='AU', mass='solar', refine=False)
nV, n0, n1, n2, tag = 355, 255, 169, 227, '_t0'
#nV, n0, n1, n2, tag = 1900, 1778, 1669, 1727, '_t5k'
snapV = simV.load_snapshot(nV)
snap0 = sim0.load_snapshot(n0)
snap1 = sim1.load_snapshot(n1)
snap2 = sim2.load_snapshot(n2)
snaplist = [snapV, snap0, snap1, snap2]
redshift = [snap.header.Redshift for snap in snaplist]

In [ ]:
rdisk = 16000 #AU
rcore = 2062648.06 #10 pc in AU

face = [[('x', 0.29518), ('z', 0.825), ('x',np.pi/2)],
        [('z', 1.16486), ('x',np.pi/2)],
        [('y', 1.87084), ('z', 2.1885), ('x',np.pi/2)],
        [('y', 0.95307), ('z', 1.5884), ('x',np.pi/2)]
       ]

disk = pd.DataFrame()
core = pd.DataFrame()
for i, snap in enumerate(snaplist):
    snap.gas.cleanup()
    print '\n'+snap.sim.name
    pos = snap.gas.get_coords(system='spherical', centering='avg', view=face[i])
    incore = np.where(snap.gas.spherical_coords[:,0] <= rcore)[0]
    print '   ', incore.size, "particles inside", rcore, snap.sim.units.length_unit
    pos = pos[incore]
    vel = snap.gas.get_velocities(system='spherical')[incore]
    mass = snap.gas.get_masses()[incore]
    dens = snap.gas.get_number_density()[incore]
    temp = snap.gas.get_temperature()[incore]
    data = np.column_stack((pos[:,0],vel[:,0],vel[:,2],mass,dens,temp))
    cols = ['radius', 'vr', 'vphi', 'mass', 'density', 'temperature']
    df = pd.DataFrame(data, columns=cols)
    df['sim'] = snap.sim.name.split('/')[1]
    core = pd.concat((core, df))
    disk = pd.concat((disk, df[df.radius <= rdisk]))
    snap.gas.cleanup()
    snap.close()

In [ ]:
grid = sns.FacetGrid(disk, hue='sim', size=8, hue_order=simlist)
grid.map(plt.scatter, 'radius', 'vr', s=10)
grid.set(xlim=(0, 15e3))
#grid.set(xlim=(0,4))#, ylim=(0,1e3))
grid.add_legend()

In [ ]:
grid = sns.FacetGrid(disk, hue='sim', size=8, hue_order=simlist)
grid.map(plt.scatter, 'radius', 'vphi', s=10)
grid.set(xlim=(0, 15e3))
#grid.set(xlim=(0,4))#, ylim=(0,1e3))
grid.add_legend()

In [ ]:
grid = sns.FacetGrid(disk, hue='sim', size=8, hue_order=simlist)
grid.map(plt.scatter, 'radius', 'density', s=10)
grid.set(xlim=(0, 15e3), yscale='log')
#grid.set(xlim=(0,4))#, ylim=(0,1e3))
grid.add_legend()

In [ ]:
grid = sns.FacetGrid(disk, hue='sim', size=8, hue_order=simlist)
grid.map(plt.scatter, 'radius', 'temperature', s=10)
grid.set(xlim=(0, 15e3))
#grid.set(xlim=(0,4))#, ylim=(0,1e3))
grid.add_legend()

In [ ]:
bigbins = np.linspace(0,16000,17)

In [ ]:
# bin the data
grouped = disk.groupby(['sim', pd.cut(disk.radius, bigbins)])
mbins = grouped.mean()
mbins.rename(columns={'radius':'r_avg'}, inplace=True)
mbins['mass'] = grouped['mass'].sum()
mbins.reset_index('sim',inplace=True)
mbins.iloc[::16]

In [ ]:
with sns.axes_style("white"):
    fig, ax = plt.subplots()

#for i, gasprop in enumerate(['vr_kms', 'vphi_kms', 'tshell', 'ndensity']):
for sim in simlist[:-1]:
    df = mbins[mbins.sim == sim]
    l, = ax.step(bigbins, np.concatenate((df.mass.values[:1], df.mass.values)), label=simname[sim], lw=2.5)
    #ax.axhline(df.Mbe.iloc[0], ls='--', color=l.get_c(), zorder=1)
    #ax.plot(midpoints, df.Mbe, color= l.get_c(), zorder=1, marker='o', ms=4)
ax.set_xlim(0,10e3)
ax.set_ylim(0,37)
ax.legend(loc=0)
ax.set_xlabel('Radius [AU]')
ax.set_ylabel('Mass [M$_{\odot}$]')
fig.savefig('figures/radial_bins/mass_bins'+tag+'.png', bbox_inches='tight')

In [ ]:
# calculate the Bonnor-Ebert mass
disk['Mbe'] = 700 * (disk.temperature/200)**1.5 / np.sqrt(disk.density/1e4)
disk['cs'] = np.sqrt(pyg.constants.k_B * disk.temperature / pyg.constants.m_H)
disk['omega'] = disk.vphi / (disk.radius * 149597871)
disk['shu_acc'] = 0.975 * disk.cs**3 / pyg.constants.GRAVITY / 1.989e33 * 3.15569e7 * 1e3

In [ ]:
diskbins = np.linspace(0,1e4,20)
rcm = diskbins * 1.49597871e13
area = np.pi * (rcm[1:]**2 - rcm[:-1]**2)
area

In [ ]:
midpoints = diskbins[:-1] + np.diff(diskbins)/2

In [ ]:
# bin the data
grouped = disk.groupby(['sim', pd.cut(disk.radius, diskbins, labels=midpoints)])
binned = grouped.mean()
binned.rename(columns={'radius':'r_avg'}, inplace=True)
binned['mass'] = grouped['mass'].sum()
binned['nbe'] = binned.mass / binned.Mbe
binned['area'] = np.concatenate([area]*4)
binned['sigma'] = binned.mass / binned.area * 1.989e33
binned['q'] = binned.cs * binned.omega / (np.pi * pyg.constants.GRAVITY * binned.sigma)
binned.reset_index('sim',inplace=True)
binned.iloc[::20]

In [ ]:
with sns.axes_style("ticks"):
    fig, axarr = plt.subplots(5, 1, sharex=True, figsize=(8,15))

for i, gasprop in enumerate(['vr', 'vphi', 'shu_acc', 'sigma', 'q']):
    for sim in simlist[:-1]:
        df = binned[binned.sim == sim]
        axarr[i].plot(midpoints,df[gasprop],
                      ls='-', label=simname[sim], marker='o', ms=3)
axarr[2].set_yscale('log')
axarr[3].set_yscale('log')
axarr[0].set_ylim(-2.4,0)
axarr[1].set_ylim(1,3.9)
#axarr[0].set_ylim(-1,1)
#axarr[1].set_ylim(0,7.9)
#axarr[2].set_ylim(80, 1.5e3)
#axarr[3].set_ylim(1e5,5e11)
axarr[-1].set_xlim(1e2, 1e4)
axarr[-1].legend(loc=0)

axarr[0].set_ylabel('Radial Velocity [km/s]')
axarr[1].set_ylabel('Rotational Velocity [km/s]')
axarr[2].set_ylabel('\.{M} \, [$10^{-3}\,$M$_{\odot}\,$yr$^{-1}$]')
axarr[2].text(.9,.9,"$0.975 \,c_s^3/G$", ha='right', va='top', size=18, transform=axarr[i].transAxes)
axarr[3].set_ylabel('Surface Density [cm$^{-2}$]')
axarr[4].set_ylabel('Q')
axarr[-1].set_xlabel("Radius [AU]")
fig.subplots_adjust(hspace=0.01)
sns.despine()
fig.savefig('figures/radial_bins/disk_profile.png', bbox_inches='tight')

In [ ]:
corebins = np.concatenate((np.zeros(1), sinkt0[sinkt0.sim == 'vanilla'].rau.values))
corebins = pd.cut(core.radius, corebins, labels = corebins[1:])

In [ ]:
# bin the data
grouped = core.groupby(['sim', corebins])
cbinned = grouped.mean()
cbinned['mass'] = grouped['mass'].sum()
cbinned['cs'] = np.sqrt(pyg.constants.k_B * cbinned.temperature / pyg.constants.m_H)
cbinned['accretion'] = 0.975 * cbinned.cs**3 / pyg.constants.GRAVITY / 1.989e33 * 3.15569e7 * 1e3
cbinned.rename(columns={'radius':'r_avg'}, inplace=True)
cbinned.reset_index('sim',inplace=True)
cbinned.head()

In [ ]:
with sns.axes_style("ticks"):
    fig, axarr = plt.subplots(3, 1, sharex=True, figsize=(5,10))

for i, gasprop in enumerate(['vphi', 'vr', 'accretion']):
    for sim in simlist[:-1]:
        df = cbinned[cbinned.sim == sim]
        axarr[i].semilogx(df.index, df[gasprop].values, marker='o', ms=3, label=simname[sim])#
    axarr[i].axvline(1e4, color='.8', zorder=1)
#axarr[2].axhline(2.725 * (redshift[3] + 1), linestyle='--', color='.5')   

axarr[0].set_ylim(-.95,3.9)
axarr[1].set_ylim(-3,0)
axarr[2].set_ylim(0, 5.9)
axarr[-1].set_xlim(100, 2e6)

axarr[1].legend(loc=0)
axarr[0].set_ylabel('Rotational Velocity [km/s]')
axarr[1].set_ylabel('Radial Velocity [km/s]')
axarr[2].set_ylabel('\.{M} \, [$10^{-3}\,$M$_{\odot}\,$yr$^{-1}$]')
axarr[2].text(.9,.9,"$0.975 \,c_s^3/G$", ha='right', va='top', size=18, transform=axarr[i].transAxes)
axarr[-1].set_xlabel("Radius [AU]")
fig.subplots_adjust(hspace=0.01)
sns.despine()
fig.savefig('figures/radial_bins/core_profile.png', bbox_inches='tight', dpi=300)